In [ ]:
# 필수 라이브러리 설치
!pip install flask pyngrok ultralytics

# pyngrok에 내 계정의 authtoken 등록
from pyngrok import conf, ngrok
conf.get_default().auth_token = "abc"  # 여기에 본인의 ngrok 토큰 입력

# 모델 로딩
from ultralytics import YOLO
model = YOLO('yolov8n.pt')  # 또는 'yolov8s.pt', 'yolov8m.pt' 등 가벼운 모델 선택 가능

# 서버 정의
from flask import Flask, request, jsonify
import base64
import numpy as np
import cv2

app = Flask(__name__)

# 카메라 파라미터 (거리 계산용)
FOCAL_LENGTH = 600
REAL_CAR_WIDTH = 1.3

@app.route('/detect', methods=['POST'])
def detect_objects():
    try:
        data = request.get_json()
        img_b64 = data['image']
        img_bytes = base64.b64decode(img_b64)

        # 바이트를 이미지로 디코딩
        nparr = np.frombuffer(img_bytes, np.uint8)
        frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        # YOLO로 객체 탐지 수행
        results = model(frame)
        response_data = []

        for result in results:
            for box in result.boxes:
                cls = int(box.cls[0])  # 클래스 ID
                if cls == 2 or cls == 7:  # 클래스: 자동차(2) 또는 트럭(7)
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    car_width = x2 - x1
                    if car_width > 0:
                        distance = (REAL_CAR_WIDTH * FOCAL_LENGTH) / car_width
                        response_data.append({
                            'class': cls,
                            'bbox': [x1, y1, x2, y2],
                            'distance': round(distance, 2)
                        })

        return jsonify({'results': response_data})

    except Exception as e:
        return jsonify({'error': str(e)}), 500

# ngrok으로 외부 접속 주소 생성
public_url = ngrok.connect(5000)
print("YOLO API 외부 접속 주소:", public_url)

# Flask 서버 실행
app.run()
